# Dokumentation af dataudtræk fra Sherpa Romeo API

Denne notebook dokumenterer udtræk af tidsskriftdata fra Sherpas Object Retrieval API.

Udtrækket omfatter JSON-data for ca. 30.000 tidsskrifter. Logfiler fra udtrækket genereres automatisk.

API'ens officielle dokumentation findes her: https://v2.sherpa.ac.uk/api/object-retrieval.html

Spørgsmål og kommentarer kan sendes til Martin Hauge Zeuner (maha@kb.dk).

### Import af afhængigheder

In [ ]:
import pandas as pd
import requests
import json
import logging
from datetime import date
from time import sleep
from pathlib import Path

### Mappeopsætning
Hentet data samles i en undermappe navngivet efter dato. Skal der laves flere kørsler på samme dag, kan `crawl_id` ændres til en anden tekststreng.

In [ ]:
crawl_id = date.strftime(date.today(), '%Y-%m-%d')

project_folder = Path('crawls', crawl_id)

Path(project_folder, 'data').mkdir(parents=True)

### Logger-opsætning

In [ ]:
logger = logging.getLogger()

logfile_path = Path(project_folder, f'{crawl_id}_sherpa.log')

file_handler = logging.FileHandler(filename=logfile_path, mode='a')
formatter = logging.Formatter('[%(asctime)s] %(levelname)s (%(module)s): %(message)s')
file_handler.setFormatter(formatter)

logger.addHandler(file_handler)
logger.setLevel(logging.DEBUG)

### Opsætning af query-parametre

API'ens endpoint defineres.

In [ ]:
endpoint = 'https://v2.sherpa.ac.uk/cgi/retrieve'

Personlig API-nøgle indlæses `config.json`.

In [ ]:
with open('config.json') as f:
    key = json.loads(f.read())['api-key']

Query-parametre gemmes i en dictionary.

In [ ]:
params = {
    'api-key': key,
    'format': 'Json',
    'item-type': 'publication',
    'limit': 100,
    'offset': 0
}

## Definition af Crawler funktion

Crawleren indlæser et antal tidsskrifter, defineret i `params['limit']`, og gemmer den returnerede JSON data i en tekstfil. Funktionen kalder sig selv rekursivt, så længe det indlæste antal tidsskrifter matcher, det anmodede antal tidsskrifter (`params['limit']`). Dvs. når det indlæste antal tidsskrifter er mindre end det anmodede antal tidsskrifter, er enden af resultatet nået.

Hver HTTP-anmodning er adskilt af minimum 1 sekunds hvile.

In [ ]:
def api_crawl(url, params):
    
    logger.debug(f'Retrieving results {params["offset"]}-{params["offset"] + params["limit"]}')
    res = requests.get(url, params=params)
    res.raise_for_status()
    
    response_json = res.json()
    
    with open(Path(project_folder, 'data', f'{crawl_id}_journals_{params["offset"]:05}.json'), 'w') as f:
        f.write(json.dumps(response_json))
    
    if len(response_json['items']) == params['limit']:
        params['offset'] += params['limit']
        
        sleep(1)
        
        api_crawl(url, params)
    else:
        logger.debug('End of results')
        

## Start Crawler

Crawleren sættes i gang med det definerede endpoint og query-parametre.

In [ ]:
api_crawl(endpoint, params)

___

## Post-processering

### Kombinér output-filer
Da API'en begrænser antallet af tidsskrifter per kald til 100, samles de enkelte output-filer endeligt til én JSON-fil.

In [ ]:
journals = []

journals_filename = Path(project_folder, f'{crawl_id}_sherpa_journals.json')

output_files = [Path(f) for f in Path(project_folder, 'data').iterdir()]

for output_file in output_files:
    with open(output_file, 'r') as f:
        journals.extend(json.loads(f.read())['items'])
        
with open(journals_filename, 'w') as f:
    f.write(f'{{"items": {json.dumps(journals)}}}')

### Sanitér log-fil
Fjern sensitive data (API-nøgle) fra log-fil.

In [ ]:
with open(logfile_path, 'r+') as f:
    log_text = f.read()
    sanitised_log_text = log_text.replace(params['api-key'], 'SECRET-KEY')
    f.seek(0)
    f.write(sanitised_log_text)
    f.truncate()